In [17]:
import tensorflow as tf
import os
import cv2
import numpy as np

In [18]:
# Set dataset path
# Load list of class names
with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/classInd.txt") as f:
    class_names = [line.strip().split()[1] for line in f.readlines()]

In [19]:
def vid_to_np_arr(filename,filepath,resized_cls_dir):
    # Load the video file
    # resized_file_path = "../Datasets/UCF101 Resized/"
    video = cv2.VideoCapture(filepath)
    frames = []
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (64,64))
        # Save the resized image
        file_path = resized_cls_dir + '/' + filename
        cv2.imwrite(file_path, frame)
        frames.append(np.array(frame))
    np_arry = np.array(frames)
    video.release()
    return np_arry

In [20]:
# Load training data
train_data = []
train_labels = []
resized_ds_dir = "../Datasets/UCF101_Resized/UCF101/UCF-101/"
for class_id, class_name in enumerate(class_names):
    class_dir = os.path.join('../Datasets/UCF101/UCF101/UCF-101/',class_name)
    os.makedirs(resized_ds_dir+class_name)
    resized_cls_dir = resized_ds_dir + class_name
    for filename in os.listdir(class_dir):
        filepath = os.path.join(class_dir, filename)
        # video = np.load(filepath)
        video = vid_to_np_arr(filename,filepath,resized_cls_dir)
        train_data.append(video)
        train_labels.append(class_id)

error: OpenCV(4.7.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:692: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'


In [ ]:
# Load testing data
test_data = []
test_labels = []
with open("../Datasets/UCF101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist/testlist01.txt") as f:
    for line in f.readlines():
        filename = line.strip().split()[0]
        class_name = filename.split("/")[0]
        class_id = class_names.index(class_name)
        filepath = os.path.join(filename)
        # video = np.load(filepath,allow_pickle=True)
        video = vid_to_np_arr(filepath)
        test_labels.append(class_id)

In [ ]:

# Convert data to tensors and normalize pixel values to range [0, 1]
train_data = tf.convert_to_tensor(train_data, dtype=tf.float32) / 255.
train_labels = tf.convert_to_tensor(train_labels, dtype=tf.int32)
test_data = tf.convert_to_tensor(test_data, dtype=tf.float32) / 255.
test_labels = tf.convert_to_tensor(test_labels, dtype=tf.int32)

# Create dataset objects
train_ds = tf.data.Dataset.from_tensor_slices((train_data, train_labels)).shuffle(len(train_data)).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((test_data, test_labels)).batch(32)

: 

: 

In [ ]:

# Define the CNN model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', input_shape=(16, 112, 112, 3)),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(128, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Conv3D(256, (3, 3, 3), activation='relu'),
    tf.keras.layers.MaxPooling3D((2, 2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(class_names), activation='softmax')
])


In [ ]:

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

# Train the model
model.fit(train_ds, epochs=10)


In [ ]:

# Evaluate the model on the test set
model.evaluate(test_ds)
